In [1]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [9]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [10]:
RDS_HOST = 'banco-db.cu1ulzl8g9j8.us-east-1.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

banco-db.cu1ulzl8g9j8.us-east-1.rds.amazonaws.com


In [11]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

##### Creamos Dimensión de Clientes

In [12]:
sql_query = 'SELECT * FROM clientes;'
df_clientes = pd.read_sql(sql_query, postgres_driver)
df_clientes.head()

id_cliente nombre apellido                      correo fecha_nacimiento  \
0    85578952  Kayla  Rosales        xcarroll@hotmail.com       1917-01-06   
1    91955737  Kelly    Colon          hdavis@hotmail.com       1958-01-04   
2    55565256  Diane  Carlson          oroberts@yahoo.com       1997-08-13   
3    98026700  Emily   Mclean   levinechelsey@hotmail.com       1997-08-30   
4    55519283  Linda   Martin  wilsonpatricia@hotmail.com       1983-01-19   

      telefono  id_direccion  id_genero  
0  578-41-9943           259       2003  
1  558-35-1302           386       2003  
2  107-61-0500           294       2003  
3  498-41-0299            56       2003  
4  475-80-0940           408       2003

In [13]:
sql_query = 'SELECT * FROM direcciones;'
df_direcciones = pd.read_sql(sql_query, postgres_driver)
df_direcciones.head()

id_direccion no_casa avenida calle  zona  id_sector
0             0   24-76       7    25     4      56565
1             1    34-3      31     9    12      58784
2             2    6-39      19    10    19      78786
3             3   80-17      21    34    14      58784
4             4   67-27      31    41    14      58889

In [14]:
df_clientes_direcciones = df_clientes.merge(df_direcciones, on='id_direccion', how='inner')
df_clientes_direcciones.head()

id_cliente nombre apellido                      correo fecha_nacimiento  \
0    85578952  Kayla  Rosales        xcarroll@hotmail.com       1917-01-06   
1    91955737  Kelly    Colon          hdavis@hotmail.com       1958-01-04   
2    55565256  Diane  Carlson          oroberts@yahoo.com       1997-08-13   
3    98026700  Emily   Mclean   levinechelsey@hotmail.com       1997-08-30   
4    55519283  Linda   Martin  wilsonpatricia@hotmail.com       1983-01-19   

      telefono  id_direccion  id_genero no_casa avenida calle  zona  id_sector  
0  578-41-9943           259       2003   91-83       5    18     1      58889  
1  558-35-1302           386       2003   57-79      31    43    17      56565  
2  107-61-0500           294       2003   14-59       4     4     4      58978  
3  498-41-0299            56       2003   15-80      27    31     1      58998  
4  475-80-0940           408       2003   91-72      42     8     7      56565

In [15]:
sql_query = 'SELECT * FROM generos;'
df_generos = pd.read_sql(sql_query, postgres_driver)
df_generos.head()

id_genero    genero
0       2002  Maculino
1       2003  Femenino

In [16]:
df_clientes_direcciones_gen = df_clientes_direcciones.merge(df_generos, how='inner', on='id_genero')
df_clientes_direcciones_gen.head()

id_cliente nombre apellido                      correo fecha_nacimiento  \
0    85578952  Kayla  Rosales        xcarroll@hotmail.com       1917-01-06   
1    91955737  Kelly    Colon          hdavis@hotmail.com       1958-01-04   
2    55565256  Diane  Carlson          oroberts@yahoo.com       1997-08-13   
3    98026700  Emily   Mclean   levinechelsey@hotmail.com       1997-08-30   
4    55519283  Linda   Martin  wilsonpatricia@hotmail.com       1983-01-19   

      telefono  id_direccion  id_genero no_casa avenida calle  zona  \
0  578-41-9943           259       2003   91-83       5    18     1   
1  558-35-1302           386       2003   57-79      31    43    17   
2  107-61-0500           294       2003   14-59       4     4     4   
3  498-41-0299            56       2003   15-80      27    31     1   
4  475-80-0940           408       2003   91-72      42     8     7   

   id_sector    genero  
0      58889  Femenino  
1      56565  Femenino  
2      58978  Femenino  
3      58998  Femenino  
4      56565  Femenino

In [17]:
sql_query = 'SELECT * FROM sectores;'
df_sectores = pd.read_sql(sql_query, postgres_driver)
df_sectores.head()

id_sector        nombre   latitud  longitud
0      58974        Centro  90.56890  91.56870
1      58978    Centro Sur  90.59810  91.56878
2      58998  Centro Norte  90.59875  91.59897
3      58784      Sur Este  90.78785  91.45780
4      78786    Norte Este  90.56890  91.56870

In [18]:
dimClientes = df_clientes_direcciones_gen.merge(df_sectores, how='inner', 
                                                                                          on='id_sector',
                                                                                          suffixes=('', '_sucursal'))
dimClientes.drop(['id_direccion', 'id_genero', 'id_sector', 'latitud', 'longitud'], axis=1,  inplace=True)
dimClientes.head()

id_cliente   nombre  apellido                        correo  \
0    85578952    Kayla   Rosales          xcarroll@hotmail.com   
1    29383808     Lisa     Allen           scott71@hotmail.com   
2    76898118   Angela     Singh         rodneyjones@gmail.com   
3    87236647    Amber  Williams  gomezchristopher@hotmail.com   
4    47450832  Melissa    Santos            ekelly@hotmail.com   

  fecha_nacimiento     telefono no_casa avenida calle  zona    genero  \
0       1917-01-06  578-41-9943   91-83       5    18     1  Femenino   
1       1929-10-27  374-26-8134   13-62      41     8    16  Femenino   
2       1927-04-17  443-22-6387   59-29      42    33    16  Femenino   
3       1932-10-14  887-28-3267   44-72      32    47     1  Femenino   
4       1915-02-02  359-62-9593   69-55       6     7    16  Femenino   

  nombre_sucursal  
0     Norte Oeste  
1     Norte Oeste  
2     Norte Oeste  
3     Norte Oeste  
4     Norte Oeste

#### Dimension de cuentas

In [19]:
sql_query = 'SELECT * FROM cuentas;'
df_cuentas = pd.read_sql(sql_query, postgres_driver)
df_cuentas.head()

id_cuenta  nombre_cuenta  balance  id_tipo_cuenta  id_cliente
0    7054053  Kayla Rosales   200.16           90036    85578952
1    1241044    Kelly Colon   472.84           90019    91955737
2    9227843  Diane Carlson   563.35           90044    55565256
3    7620752   Emily Mclean   450.41           90026    98026700
4    7957427   Linda Martin   463.56           90036    55519283

In [20]:
sql_query = 'SELECT * FROM tipos_cuentas;'
df_tipoCuentas = pd.read_sql(sql_query, postgres_driver)
df_tipoCuentas.head()

id_tipo_cuenta          tipo_cuenta
0           90025  Monetaria Quetzales
1           90026    Monetaria Dolares
2           90036     Ahorro Quetzales
3           90078       Ahorro Dolares
4           90044    Cheques Quetzales

In [21]:
dfCuentas = df_cuentas.merge(df_tipoCuentas, how='inner', on='id_tipo_cuenta')
dimCuentas = dfCuentas.drop(['id_tipo_cuenta'], axis=1)
dimCuentas.head()

id_cuenta      nombre_cuenta  balance  id_cliente       tipo_cuenta
0    7054053      Kayla Rosales   200.16    85578952  Ahorro Quetzales
1    7957427       Linda Martin   463.56    55519283  Ahorro Quetzales
2    8428488     Valerie Barker   196.60    37925742  Ahorro Quetzales
3    7215755      Melissa Brown   159.80    25747227  Ahorro Quetzales
4    7828639  Courtney Campbell   203.54    88021991  Ahorro Quetzales

##### Dimensión de tipoTransacciones

In [22]:
sql_query = 'SELECT * FROM tipo_transacciones;'
dimTipoTransacciones = pd.read_sql(sql_query, postgres_driver)
dimTipoTransacciones.head()

id_tipo_transac tipo_transaccion
0            85095          Depóito
1            85098           Retiro
2            85194    Transferencia
3            85133    Pago Prestamo

##### Dimensión de Divisas

In [23]:
sql_query = 'SELECT * FROM divisas;'
dimDivisas = pd.read_sql(sql_query, postgres_driver)
dimDivisas.head()

id_divisa nombre_divisa simbolo
0     103589     Quetzales       Q
1     114589       Dolares       $
2     125487         Euros       €

##### Dimensión de Sucursales

In [24]:
sql_query = 'SELECT * FROM sucursales;'
df_sucursales = pd.read_sql(sql_query, postgres_driver)
df_sucursales.head()

id_sucursal                  nombre  id_sector  id_gerente
0       187301   Jennifer Springs Apt.      58998       28539
1       182470               Ray Manor      58998       98832
2       186408  Perkins Causeway Suite      58889       22612
3       188366               Unit  Box      78786       17767
4       188213          Roberts Stream      58784       15106

In [25]:
sql_query = 'SELECT * FROM sectores;'
df_sectores = pd.read_sql(sql_query, postgres_driver)
df_sectores.head()

id_sector        nombre   latitud  longitud
0      58974        Centro  90.56890  91.56870
1      58978    Centro Sur  90.59810  91.56878
2      58998  Centro Norte  90.59875  91.59897
3      58784      Sur Este  90.78785  91.45780
4      78786    Norte Este  90.56890  91.56870

In [26]:
df_sucursales_sectores = df_sucursales.merge(df_sectores, on='id_sector',
                                                                                                         how='inner',
                                                                                                         suffixes=('_sucursal', '_sector'))
df_sucursales_sectores.head()

id_sucursal         nombre_sucursal  id_sector  id_gerente nombre_sector  \
0       187301   Jennifer Springs Apt.      58998       28539  Centro Norte   
1       182470               Ray Manor      58998       98832  Centro Norte   
2       186666     Casey Heights Suite      58998       37414  Centro Norte   
3       186237          Compton Summit      58998       63642  Centro Norte   
4       186408  Perkins Causeway Suite      58889       22612   Norte Oeste   

    latitud  longitud  
0  90.59875  91.59897  
1  90.59875  91.59897  
2  90.59875  91.59897  
3  90.59875  91.59897  
4  90.78745  91.78897

In [27]:
sql_query = 'SELECT * FROM colaboradores;'
df_colaboradores = pd.read_sql(sql_query, postgres_driver)
df_colaboradores.head()

id_colaborador   nombre   apellido  id_cargo
0           26618      Roy   Peterson       101
1           30188    Brian    Serrano       202
2           74527    Laura    Anthony       101
3           34304    Debra     Graves       101
4           15251  Jeffrey  Patterson       101

In [28]:
df_suc_sec_manager = df_sucursales_sectores.merge(df_colaboradores, 
                                                                                               left_on='id_gerente',
                                                                                               right_on='id_colaborador',
                                                                                               how='inner')
df_suc_sec_manager['nombre_gerente'] = df_suc_sec_manager['nombre'] + ' ' + df_suc_sec_manager['apellido']
dimSucursal = df_suc_sec_manager.drop(['id_sector', 'id_gerente', 'latitud', 'id_colaborador' ,'longitud', 'nombre', 'apellido', 'id_cargo'], axis=1)
dimSucursal.head()

id_sucursal         nombre_sucursal nombre_sector      nombre_gerente
0       187301   Jennifer Springs Apt.  Centro Norte           Kara Ryan
1       182470               Ray Manor  Centro Norte  Cassandra Matthews
2       186666     Casey Heights Suite  Centro Norte      Patricia Vance
3       186237          Compton Summit  Centro Norte       Stanley Stone
4       186408  Perkins Causeway Suite   Norte Oeste       Tommy Sanchez

##### Dimensión de Colaboradores

In [29]:
sql_query = 'SELECT * FROM colaboradores;'
df_colaboradores = pd.read_sql(sql_query, postgres_driver)
df_colaboradores.head()

id_colaborador   nombre   apellido  id_cargo
0           26618      Roy   Peterson       101
1           30188    Brian    Serrano       202
2           74527    Laura    Anthony       101
3           34304    Debra     Graves       101
4           15251  Jeffrey  Patterson       101

In [30]:
sql_query = 'SELECT * FROM cargos;'
df_cargos = pd.read_sql(sql_query, postgres_driver)
df_cargos.head()

id_cargo          titulo_cargo
0       101     Cajero - Receptor
1       202   Gerente de Sucursal
2       303  Supervisor de Sector

In [31]:
dfColaboradores = df_colaboradores.merge(df_cargos, how='inner', on='id_cargo')
dimColaboradores = dfColaboradores.drop(['id_cargo'], axis=1)
dimColaboradores.head()

id_colaborador   nombre    apellido       titulo_cargo
0           26618      Roy    Peterson  Cajero - Receptor
1           74527    Laura     Anthony  Cajero - Receptor
2           34304    Debra      Graves  Cajero - Receptor
3           15251  Jeffrey   Patterson  Cajero - Receptor
4           86428     Carl  Cunningham  Cajero - Receptor

##### Dimensión de Tiempo 

In [32]:
sql_query = 'SELECT fecha_hora FROM transacciones;'
dimFechaHora = pd.read_sql(sql_query, postgres_driver)
dimFechaHora.head()

fecha_hora
0 2023-03-19 22:11:33
1 2023-03-08 13:45:08
2 2023-03-05 15:58:51
3 2023-01-23 23:19:38
4 2023-03-06 11:31:03

In [33]:
dimFechaHora['year'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).year
dimFechaHora['month'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).month
dimFechaHora['quarter'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).quarter
dimFechaHora['day'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).day
dimFechaHora['week'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).week
dimFechaHora['dayofweek'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).dayofweek
dimFechaHora['hour'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).hour
dimFechaHora['minute'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).minute
dimFechaHora['is_weekend'] = dimFechaHora['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimFechaHora.head()

/var/folders/52/w3v5_lr57951tzsccv4jk6fw0000gn/T/ipykernel_83444/2466271033.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFechaHora['week'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).week


fecha_hora  year  month  quarter  day  week  dayofweek  hour  \
0 2023-03-19 22:11:33  2023      3        1   19    11          6    22   
1 2023-03-08 13:45:08  2023      3        1    8    10          2    13   
2 2023-03-05 15:58:51  2023      3        1    5     9          6    15   
3 2023-01-23 23:19:38  2023      1        1   23     4          0    23   
4 2023-03-06 11:31:03  2023      3        1    6    10          0    11   

   minute  is_weekend  
0      11           1  
1      45           0  
2      58           1  
3      19           0  
4      31           0

In [34]:
dimFechaHora['id_fecha'] = dimFechaHora['year'].astype(str) + dimFechaHora['month'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['day'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['hour'].astype(str)
dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['minute'].astype(str)
dimFechaHora.drop_duplicates(inplace=True)
dimFechaHora.head()

fecha_hora  year  month  quarter  day  week  dayofweek  hour  \
0 2023-03-19 22:11:33  2023      3        1   19    11          6    22   
1 2023-03-08 13:45:08  2023      3        1    8    10          2    13   
2 2023-03-05 15:58:51  2023      3        1    5     9          6    15   
3 2023-01-23 23:19:38  2023      1        1   23     4          0    23   
4 2023-03-06 11:31:03  2023      3        1    6    10          0    11   

   minute  is_weekend     id_fecha  
0      11           1  20233192211  
1      45           0   2023381345  
2      58           1   2023351558  
3      19           0  20231232319  
4      31           0   2023361131

#### Tabla de Hechos

In [35]:
sql_query = '''SELECT id_transaccion, fecha_hora, id_tipo_transaccion
                              id_divisa, id_cliente, id_colaborador, id_sucursal, id_cuenta_origen, id_cuenta_destino,
                              monto FROM transacciones;'''
df_factTable = pd.read_sql(sql_query, postgres_driver)
df_factTable.head()

id_transaccion          fecha_hora  id_divisa  id_cliente  id_colaborador  \
0               0 2023-03-19 22:11:33      85095    65407986           62646   
1               1 2023-03-08 13:45:08      85133    55278135           56836   
2               2 2023-03-05 15:58:51      85095    95955468           88705   
3               3 2023-01-23 23:19:38      85133    16067625           28539   
4               4 2023-03-06 11:31:03      85095    47251515           71835   

   id_sucursal  id_cuenta_origen  id_cuenta_destino    monto  
0       189141           9324817            4493450  8787.19  
1       185657           2320881            9126474  4700.10  
2       181035           2571055            3320992  9849.36  
3       180638           3485409            1067228  4367.44  
4       188213           3655299            5652533  4450.03

In [36]:
df_factTable['year'] = pd.DatetimeIndex(df_factTable['fecha_hora']).year
df_factTable['month'] = pd.DatetimeIndex(df_factTable['fecha_hora']).month
df_factTable['day'] = pd.DatetimeIndex(df_factTable['fecha_hora']).day
df_factTable['hour'] = pd.DatetimeIndex(df_factTable['fecha_hora']).hour
df_factTable['minute'] = pd.DatetimeIndex(df_factTable['fecha_hora']).minute
df_factTable['id_fecha'] = df_factTable['year'].astype(str) + df_factTable['month'].astype(str)
df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['day'].astype(str)
df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['hour'].astype(str)
df_factTable['id_fecha'] = df_factTable['id_fecha'].astype(str) + df_factTable['minute'].astype(str)
factTable_transacciones = df_factTable.drop(['year', 'month', 'day', 'hour', 'minute', 'fecha_hora'], axis=1)
factTable_transacciones.head()

id_transaccion  id_divisa  id_cliente  id_colaborador  id_sucursal  \
0               0      85095    65407986           62646       189141   
1               1      85133    55278135           56836       185657   
2               2      85095    95955468           88705       181035   
3               3      85133    16067625           28539       180638   
4               4      85095    47251515           71835       188213   

   id_cuenta_origen  id_cuenta_destino    monto     id_fecha  
0           9324817            4493450  8787.19  20233192211  
1           2320881            9126474  4700.10   2023381345  
2           2571055            3320992  9849.36   2023351558  
3           3485409            1067228  4367.44  20231232319  
4           3655299            5652533  4450.03   2023361131

##### Lectura de Archivo desde S3

In [37]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [38]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

mibucket2023


In [39]:
S3_BUCKET_NAME = 'mibucket2023'

In [40]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['divisas/', 'divisas/conversion_divisas.xlsx']

##### leemos data de S3

In [41]:
import io

for remoteFile in remoteFileList:
    try:
        file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
        data = file['Body'].read()
        df_valor_divisas = pd.read_excel(io.BytesIO(data), engine='openpyxl')
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df_valor_divisas.head()

No es un archivo.
File is not a zip file


Mondeda_A  Mondeda_B    Cambio
0    Dolares  Quetzales  7.830000
1  Quetzales    Dolares  0.127714
2  Quetzales      Euros  0.118765
3      Euros  Quetzales  8.420000
4      Euros    Dolares  1.080000

##### Agregamos calculo de transaccion en quetzales

In [42]:
sql_query = '''SELECT id_divisa, monto FROM transacciones;'''
df_montos = pd.read_sql(sql_query, postgres_driver)
df_montos.head()

id_divisa    monto
0     114589  8787.19
1     114589  4700.10
2     125487  9849.36
3     125487  4367.44
4     114589  4450.03

In [43]:
sql_query = '''SELECT * FROM divisas;'''
df_divisas = pd.read_sql(sql_query, postgres_driver)
df_divisas.head()

id_divisa nombre_divisa simbolo
0     103589     Quetzales       Q
1     114589       Dolares       $
2     125487         Euros       €

In [44]:
df_montos = df_montos.merge(df_divisas, on='id_divisa', how='inner')
df_montos.head()

id_divisa    monto nombre_divisa simbolo
0     114589  8787.19       Dolares       $
1     114589  4700.10       Dolares       $
2     114589  4450.03       Dolares       $
3     114589  7108.67       Dolares       $
4     114589  5090.25       Dolares       $

In [45]:
df_montos['nombre_divisa'].value_counts()

Quetzales    574
Dolares      560
Euros        560
Name: nombre_divisa, dtype: int64

##### Ralizamos Calculos para Conversiones

In [46]:
montos_resultantes = []
for index, df_row in df_montos.iterrows():
     if(df_row['nombre_divisa'] != 'Quetzales'):
          factor_conversion = df_valor_divisas[
                    ((df_valor_divisas['Mondeda_A']== df_row['nombre_divisa']) &
                    (df_valor_divisas['Mondeda_B']== 'Quetzales'))]
          montos_resultantes.append(round(factor_conversion['Cambio'].values[0]*df_row['monto'], 2))
     else:
          montos_resultantes.append(df_row['monto'])
montos_resultantes[0:5]

[68803.7, 36801.78, 34843.73, 55660.89, 39856.66]

##### Agregamos columna con nuevos calculos

In [47]:
len(montos_resultantes)

1694

In [48]:
factTable_transacciones.shape

(1694, 9)

In [49]:
factTable_transacciones['monto_quetzales'] = montos_resultantes
factTable_transacciones.head()

id_transaccion  id_divisa  id_cliente  id_colaborador  id_sucursal  \
0               0      85095    65407986           62646       189141   
1               1      85133    55278135           56836       185657   
2               2      85095    95955468           88705       181035   
3               3      85133    16067625           28539       180638   
4               4      85095    47251515           71835       188213   

   id_cuenta_origen  id_cuenta_destino    monto     id_fecha  monto_quetzales  
0           9324817            4493450  8787.19  20233192211         68803.70  
1           2320881            9126474  4700.10   2023381345         36801.78  
2           2571055            3320992  9849.36   2023351558         34843.73  
3           3485409            1067228  4367.44  20231232319         55660.89  
4           3655299            5652533  4450.03   2023361131         39856.66

##### Creación de Instancias en AWS para DW

In [50]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [51]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')
print(aws_conn)

In [52]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'dw-db']


In [53]:
rdsIdentifier = 'dw-db'

In [54]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


##### Antes de conectarnos recordemos esperar el tiempo suficiente para agregar la nueva instancia y agregar el puerto 3306 al grupo de seguridad.

In [55]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw-db.cu1ulzl8g9j8.us-east-1.rds.amazonaws.com


##### Recordemos instalar MySQLdb

In [57]:
import create_dw_query
import mysql.connector as mysqlC
try:
    myDw = mysqlC.connect(
    host=RDS_DW_HOST, 
    user=config.get('RDS_MYSQL', 'DB_USER'),
    password=config.get('RDS_MYSQL', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL', 'DB_NAME')
    )

    mycursor = myDw.cursor()
    mycursor.execute(create_dw_query.CREATE_DW, multi=True)
    myDw.commit()
    print("Data Warehouse Creado Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Data Warehouse Creado Exitosamente


##### Insertamos Data

In [61]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""  

In [ ]:
#insertamos clientes.
dimClientes.to_sql('dimClientes', mysql_driver, index=False, if_exists='append')

In [64]:
#insertamos cuentas
dimCuentas.to_sql('dimCuentas', mysql_driver, index=False, if_exists='append')

505

In [66]:
dimDivisas.to_sql('dimDivisas', mysql_driver, 
                  index=False, if_exists='append', method='multi')

3

In [67]:
dimSucursal.to_sql('dimSucursales', mysql_driver, 
                   index=False, if_exists='append', method='multi')

22

In [68]:
dimColaboradores.to_sql('dimColaboradores', mysql_driver, 
                    index=False, if_exists='append', method='multi')

107

In [70]:
dimTipoTransacciones

id_tipo_transac tipo_transaccion
0            85095          Depóito
1            85098           Retiro
2            85194    Transferencia
3            85133    Pago Prestamo

In [ ]:
dimTipoTransacciones.to_sql('dimTipoTransacciones', 
                            mysql_driver, index=False, if_exists='append', method='multi')

In [ ]:
dimTipoTransacciones.to_sql('dimTipoTransacciones', 
                            mysql_driver, index=False, if_exists='append', method='multi')